# neural ODE demo

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchdiffeq import odeint

# Define ODE
class ODEFunc(nn.Module):
    def __init__(self):
        super(ODEFunc, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(784, 256),
            nn.Tanh(),
            nn.Linear(256, 784)
        )

    def forward(self, t, h):
        return self.net(h)

# Define Neural ODE model
class NeuralODEModel(nn.Module):
    def __init__(self, ode_func):
        super(NeuralODEModel, self).__init__()
        self.ode_func = ode_func
        self.fc = nn.Linear(784, 10)

    def forward(self, x):
        h0 = x.view(x.size(0), -1)  
        t = torch.tensor([0.0, 1.0]).to(x.device)  
        hT = odeint(self.ode_func, h0, t, method='dopri5')[-1]
        return self.fc(hT)

# Load data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Init model, loss func, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ode_func = ODEFunc().to(device)
model = NeuralODEModel(ode_func).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(10):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
